In [1]:
class CFG:
  train_df='./train/train_df.csv'
  test_df='./test/test_df.csv'
  target_cols=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
  classes=5
  model='microsoft/deberta-v3-base'
  embedd_dim=768
  criterion = 'mse' # ['crossentropy', 'mse', 'l1', 'focal']
  main_metric = 'acc@1'
  model_file = './models/ed_best_deberta_base.pt'
  # just use it
  apex=True
  gradient_checkpointing=True
  num_cycles=0.5
  num_warmup_steps=0
  epochs=10
  encoder_lr=2e-5
  decoder_lr=2e-5
  min_lr=1e-6
  eps=1e-6
  betas=(0.9, 0.999)
  batch_size=2
  max_len=512
  weight_decay=0.01
  # gradient_accumulation_steps=1
  max_grad_norm=1000
  seed=42
  scheduler='cosine' # ['linear', 'cosine']
  batch_scheduler=True
  #
  colab=False

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
if CFG.colab:
  from google.collab import drive
  drive.mount('/content/drive')
  import os
  os.chdir('/content/drive/MyDrive/lab/bert_finetune')

# Load libraries and data

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from tqdm import tqdm
import gc

In [5]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available(): # для GPU отдельный seed
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(CFG.seed)
# есть стохастические операции на GPU
# сделаем их детерминированными для воспроизводимости
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

In [6]:
CFG.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.model, use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
train_df = pd.read_csv(CFG.train_df, index_col='index')
test_df = pd.read_csv(CFG.test_df, index_col='index')

In [8]:
train_df.head()

,text,0,1,2,3,4,5
index,,,,,,,
1,Mortar assault leaves at least 18 dead,22,2,60,0,64,0
2,Goal delight for Sheva,0,0,0,93,0,38
3,Nigeria hostage feared dead is freed,18,0,52,66,20,65
4,Bombers kill shoppers,66,39,94,0,86,0
5,"Vegetables, not fruit, slow brain decline",0,0,25,26,2,46


In [9]:
df = pd.concat((train_df, test_df))
df.columns = ['text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
df = df.drop('Disgust', axis=1)

In [10]:
df[CFG.target_cols] = df[CFG.target_cols].apply(lambda it: it/it.sum() if it.sum() else 0, axis=1)
df.head()

,text,Anger,Fear,Joy,Sadness,Surprise
index,,,,,,
1,Mortar assault leaves at least 18 dead,0.150685,0.410959,0.000000,0.438356,0.000000
2,Goal delight for Sheva,0.000000,0.000000,0.709924,0.000000,0.290076
3,Nigeria hostage feared dead is freed,0.081448,0.235294,0.298643,0.090498,0.294118
4,Bombers kill shoppers,0.268293,0.382114,0.000000,0.349593,0.000000
5,"Vegetables, not fruit, slow brain decline",0.000000,0.252525,0.262626,0.020202,0.464646


In [11]:
tmp, valid_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, test_df = train_test_split(tmp, test_size=2/8, random_state=42)

In [21]:
ds = df.iloc[:, -5:].values

In [27]:
ds.mean(axis=0)

array([0.10100774, 0.16337742, 0.28517377, 0.20631656, 0.24092451])

In [26]:
np.unique(ds.argmax(axis=-1), return_counts=True)

(array([0, 1, 2, 3, 4]), array([110, 198, 442, 275, 225]))

# Dataset

In [12]:
class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = (df['text']).values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def get_text(self, idx):
        # tokenization
        inputs = self.cfg.tokenizer.encode_plus(
            self.texts[idx], 
            return_tensors=None, 
            add_special_tokens=True, 
            max_length=CFG.max_len,
            pad_to_max_length=True,
            # padding='longest',
            truncation=True
        )
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
        return inputs
    
    def get_labels(self, idx):
        if CFG.criterion != 'crossentropy' and CFG.criterion != 'focal':
           return torch.tensor(self.labels[idx], dtype=torch.float)
        return torch.tensor(self.labels[idx]).type(torch.LongTensor)

    def __getitem__(self, idx):
        inputs = self.get_text(idx)
        label = self.get_labels(idx)
        return inputs, label

def collate(inputs):
		# reduce sequence length
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

# Model

In [13]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [14]:
class TransformerModel(nn.Module):

    def __init__(self):
        super(TransformerModel, self).__init__()

        self.model = AutoModel.from_pretrained(CFG.model)
        if CFG.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.clf =  nn.Linear(CFG.embedd_dim, CFG.classes)
        if not CFG.criterion == 'crossentropy':
            self.sm = nn.Softmax(dim=-1)
        torch.nn.init.xavier_uniform_(self.clf.weight)

    def forward(self, input_id, mask):
        # sequence has [CLF] token in the beginning
        # bert() returns first vector as pooling of sentence
        x = self.model(input_ids= input_id, attention_mask=mask)[0]
        out = self.pool(x, mask)
        out = self.clf(out)
        if not CFG.criterion == 'crossentropy':
            return self.sm(out)
        return out

# Train loop and metrics and other

In [15]:
from scipy.stats import wasserstein_distance, pearsonr
from sklearn.metrics import f1_score, accuracy_score

In [16]:
def calculate_metrics(y, pred):
    APd = torch.mean(torch.tensor([pearsonr(pred[i], y[i])[0] for i in range(pred.size()[0])]))
    APe = torch.mean(torch.tensor([pearsonr(pred[:, i], y[:, i])[0] for i in range(CFG.classes)]))
    RMSED = torch.mean(torch.tensor([np.sqrt(nn.functional.mse_loss(pred[i:i+1, :], y[i:i+1, :])) for i in range(pred.size()[0])]))
    WD = torch.mean(torch.tensor([wasserstein_distance(pred[i], y[i]) for i in range(pred.size()[0])]))

    y = torch.argmax(y, dim=1)
    pred = torch.argmax(pred, dim=1)
    acc_1 = accuracy_score(y, pred)


    return {'acc@1': acc_1, 'APd': APd, 'APe': APe, 'RMSED': RMSED, 'WD': WD}

In [17]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "model" not in n],
         'lr': decoder_lr, 'weight_decay': 0.0}
    ]
    return optimizer_parameters

def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
        )
    return scheduler

In [18]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input, target):
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = torch.exp(logpt)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.reduction == 'mean': 
          return loss.mean()
        elif self.reduction == 'sum':
          return loss.sum()
        else:
          raise NotImplementedError(f'Not implemented reduction: {self.reduction}')

In [19]:
train_ds = TrainDataset(CFG, train_df)
valid_ds = TrainDataset(CFG, valid_df)

In [20]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_ds, batch_size=CFG.batch_size, shuffle=False)

In [21]:
model = TransformerModel()

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
if CFG.criterion == 'crossentropy':
  criterion = nn.CrossEntropyLoss()
elif CFG.criterion == 'mse':
  criterion = nn.MSELoss()
elif CFG.criterion == 'l1':
  criterion = nn.SmoothL1Loss()
elif CFG.criterion == 'focal':
  criterion = FocalLoss(5)
else:
  raise NotImplementedError('Change loss')

In [23]:
optimizer_parameters = get_optimizer_params(model,
                                           encoder_lr=CFG.encoder_lr, 
                                           decoder_lr=CFG.decoder_lr,
                                           weight_decay=CFG.weight_decay)
optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

In [24]:
num_train_steps = int(len(train_df) / CFG.batch_size * CFG.epochs)
scheduler = get_scheduler(CFG, optimizer, num_train_steps)

In [25]:
model.to(CFG.device)
criterion.to(CFG.device)

MSELoss()

In [26]:
def train_loop(model, optimizer, criterion, train_loader, valid_loader, epochs, scheduler):
  best_score = 0
  # multiplies gradient so it won't vanish (torch use float16)
  scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
  for e in range(epochs):
    # train
    model.train()
    train_loss = []
    for inputs, labels in tqdm(train_loader):
      inputs = collate(inputs)
      # move inputs to device
      mask = inputs['attention_mask'].to(CFG.device)
      input_id = inputs['input_ids'].squeeze(1).to(CFG.device)
      labels = labels.to(CFG.device)
      # forward
      with torch.cuda.amp.autocast(enabled=CFG.apex):
        y_preds = model(input_id, mask)
        loss = criterion(y_preds, labels)
      # calculate loss
      train_loss.append(loss.detach().cpu().item())

      optimizer.zero_grad()
      # loss.backward()
      scaler.scale(loss).backward()
      # gradient clipping
      grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
      scaler.step(optimizer)
      scaler.update()
      # optimizer.step()
      if CFG.batch_scheduler:
          scheduler.step()
    train_loss = np.mean(train_loss)
    # valid
    model.eval()
    with torch.no_grad():
      valid_loss = []
      preds = []
      y = []
      for inputs, labels in valid_loader:
        inputs = collate(inputs)
        # move inputs to device
        mask = inputs['attention_mask'].to(CFG.device)
        input_id = inputs['input_ids'].squeeze(1).to(CFG.device)
        labels = labels.to(CFG.device)
        # forward
        with torch.cuda.amp.autocast(enabled=CFG.apex):
          y_preds = model(input_id, mask)
          loss = criterion(y_preds, labels)
        preds.append(y_preds.detach().cpu())
        y.append(labels.detach().cpu())
        # calculate loss
        valid_loss.append(loss.detach().cpu().item())
    valid_loss = np.mean(valid_loss)
    preds = torch.concat(preds, dim=0)
    y = torch.concat(y, dim=0)
    metrics = calculate_metrics(y, preds)
    # save best model
    if best_score < metrics[CFG.main_metric]:
      torch.save(model.state_dict(), CFG.model_file)
      best_score = metrics[CFG.main_metric]
    print('best_score =', best_score)
    print(f'EPOCH {e + 1}:, train_loss = {train_loss: .5f}, valid_loss = {valid_loss}', *[f'{name} = {value: .5f}' for name, value in metrics.items()])
    

In [27]:
torch.cuda.empty_cache()
train_loop(model, optimizer, criterion, train_loader, valid_loader, CFG.epochs, scheduler)

100%|██████████| 188/188 [00:29<00:00,  6.34it/s]


best_score = 0.496
EPOCH 1:, train_loss =  0.04700, valid_loss = 0.03136499774538808 acc@1 =  0.49600 APd =  0.56061 APe =  0.54802 RMSED =  0.15600 WD =  0.08702


100%|██████████| 188/188 [00:26<00:00,  7.06it/s]


best_score = 0.556
EPOCH 2:, train_loss =  0.02435, valid_loss = 0.03200747556097451 acc@1 =  0.55600 APd =  0.59740 APe =  0.60502 RMSED =  0.15488 WD =  0.08594


100%|██████████| 188/188 [00:26<00:00,  7.07it/s]


...

best_score = 0.608
EPOCH 8:, train_loss =  0.00607, valid_loss = 0.02575140152983959 acc@1 =  0.59600 APd =  0.67433 APe =  0.65959 RMSED =  0.13637 WD =  0.07708


100%|██████████| 188/188 [00:27<00:00,  6.74it/s]


best_score = 0.608
EPOCH 9:, train_loss =  0.00560, valid_loss = 0.02605147673083203 acc@1 =  0.59600 APd =  0.67175 APe =  0.65846 RMSED =  0.13714 WD =  0.07739


100%|██████████| 188/188 [00:27<00:00,  6.73it/s]


best_score = 0.608
EPOCH 10:, train_loss =  0.00552, valid_loss = 0.025897975205369884 acc@1 =  0.59200 APd =  0.67317 APe =  0.65870 RMSED =  0.13669 WD =  0.07706


# Test

In [28]:
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
      test_loss = []
      preds = []
      y = []
      for step, (inputs, labels) in enumerate(test_loader):
        inputs = collate(inputs)
        # move inputs to device
        mask = inputs['attention_mask'].to(CFG.device)
        input_id = inputs['input_ids'].squeeze(1).to(CFG.device)
        labels = labels.to(CFG.device)
        # forward
        with torch.cuda.amp.autocast(enabled=CFG.apex):
          y_preds = model(input_id, mask)
          loss = criterion(y_preds, labels)
        preds.append(y_preds.detach().cpu())
        y.append(labels.detach().cpu())
        # calculate loss
        test_loss.append(loss.detach().cpu().item())
    test_loss = np.mean(test_loss)
    preds = torch.concat(preds, dim=0)
    y = torch.concat(y, dim=0)
    metrics = calculate_metrics(y, preds)

    print(f'Test metrics: test_loss={test_loss: 0.5f}', *[f'{name} = {value: 0.5f}' for name, value in metrics.items()])

In [29]:
model.load_state_dict(torch.load(CFG.model_file))

<All keys matched successfully>

In [30]:
test_ds = TrainDataset(CFG, test_df)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=CFG.batch_size, shuffle=False)

In [31]:
test(model, valid_loader)

Test metrics: test_loss= 0.02536 acc@1 =  0.60800 APd =  0.67336 APe =  0.66254 RMSED =  0.13606 WD =  0.07727


In [32]:
test(model, test_loader)

Test metrics: test_loss= 0.02274 acc@1 =  0.66000 APd =  0.70952 APe =  0.69687 RMSED =  0.12617 WD =  0.07235
